In [1]:
import pandas as pd

from queue import deque
from heapq import heappop, heappush

from collections import namedtuple, defaultdict

In [2]:
wiki_pages = pd.read_csv('./simple_english_wiki/simple_english_wiki_pages.csv')
wiki_pagelinks = pd.read_csv('./simple_english_wiki/simple_english_wiki_pagelinks.csv')

In [3]:
wiki_df = wiki_pages.merge(wiki_pagelinks, how='right', left_on='page_id', right_on='pl_from')
wiki_df

,page_id,page_title,pl_from,pl_title,pl_to
0,1,April,1,1320,39168
1,7560,October_24,7560,1320,39168
2,7873,October_12,7873,1320,39168
3,8393,July_20,8393,1320,39168
4,8719,April_17,8719,1320,39168
...,...,...,...,...,...
6724923,836310,Khongjom_War_Memorial_Complex,836310,Anglo-Manipuri_War,836099
6724924,836311,Anglo-Manipur_War,836311,Anglo-Manipuri_War,836099
6724925,836313,Maibam_Lokpa_Ching,836313,Battle_of_Imphal,836107
6724926,836316,Alfredo_Borrero_Vega,836316,Alfredo_Borrero,834696


In [4]:
wiki_df.page_title = wiki_df.page_title.astype(str)
wiki_df.pl_title = wiki_df.pl_title.astype(str)

In [5]:
initial_link = 'Analytics'
final_link = 'Algorithm'

In [6]:
edge = namedtuple('edge', 'link path')

In [7]:
class GraphCreation:
    def __init__(self):
        self.neighbor_links = defaultdict(set)
        self.link = set()
        
    def add_edge(self, edge):
        self.neighbor_links[edge.link].add(edge.path)
        self.link.add(edge.link)
        self.link.add(edge.path)
            
    def __getitem__(self, item):
        return self.neighbor_links.get(item)

In [8]:
graph = GraphCreation()
for i in range(len(wiki_df)):
    graph.add_edge(edge(wiki_df.page_title[i], wiki_df.pl_title[i]))

### Задача №1  
Какое наименьшее количество переходов по ссылкам нужно сделать, чтобы прийти от статьи Analytics к статье Algorithm?

In [9]:
def first_task(graph, link):
    viewed_links = set()
    links = deque()
    links.append((link, 0))
    result = list()
    while links:
        current_link, current_level = links.popleft()
        if graph[current_link] is None:
            continue
        else:
            inherited_links = [edge for edge in graph[current_link]]
        for inherited_link in inherited_links:
            if inherited_link not in viewed_links:
                viewed_links.add(inherited_link)
                inherited_level = (inherited_link, current_level+1)
                links.append(inherited_level)
                result.append(inherited_level)
                
    return result

In [10]:
print('Ответ:', [i[1] for i in first_task(graph, initial_link) if (i[0] == final_link)][0])

Ответ: 4


## Задача №2   
Какая статья на кратчайшем пути из предыдущей задачи содержит ссылку на статью Algorithm?

In [11]:
def second_task(graph, link):
    viewed_links = set()
    links = deque()
    links.append((link, 0))
    result = 0
    while links:
        current, current_level = links.popleft()
        if (graph[current] is None) or (final_link not in graph[current]):
            return result
        else:
            result += 1
            return result

In [12]:
links = list()
for i in first_task(graph, initial_link):
    if i[1] == 3:
        links.append(i[0])

In [13]:
flg = list()
for i in links:
    flg.append(second_task(graph, i))

In [14]:
result = pd.DataFrame(zip(links, flg), columns = ['links', 'flg'])
print('Ответ:', result[result.flg == 1].links.values[0])

Ответ: Serial_number


## Задача №3   
Задача такая же, как в шаге 2, но теперь мы еще и учитываем длину ссылки.    
**Например:**  
Путь Cooking -> Wood(4) -> Building(8) -> Concrete(8) имеет длину 20, а путь Cooking -> Oven(4) -> Brick(5) -> Concrete(8) имеет длину 17.

In [15]:
weighted_edge = namedtuple('edge', 'link path length')

In [16]:
class WeightedGraph:
    def __init__(self):
        self.neighbor_links = defaultdict(set)
        self.link = set()
        
    def add_edge(self, edge):
        self.neighbor_links[edge.link].add(edge)
        self.link.add(edge.link)
        self.link.add(edge.path)
            
    def __getitem__(self, item):
        return self.neighbor_links.get(item, [])

In [17]:
weighted_graph = WeightedGraph()
for i in range(len(wiki_df)):
    weighted_graph.add_edge(weighted_edge(wiki_df.page_title[i], wiki_df.pl_title[i], len(wiki_df.pl_title[i])))

In [18]:
def third_task(graph, link):
    viewed_links = set()
    links = [(0, link)]
    min_lengths = {link: 0}
    while links: 
        current_length, current_link = heappop(links)
        if current_link not in viewed_links:
            for _, next_link, length in graph.neighbor_links.get(current_link, []): 
                if next_link in viewed_links:
                    continue 
                old_min_length = min_lengths.get(next_link)
                new_length = current_length + length
                if (old_min_length is None) or (old_min_length > new_length):
                    min_lengths[next_link] = new_length
                    heappush(links, (new_length, next_link))       
            viewed_links.add(current_link)
    return min_lengths

In [19]:
print('Ответ:', third_task(weighted_graph, initial_link)[final_link])

Ответ: 29


## Задача №4   
Аналогично заданию 3 - нужно написать название статьи, которая ведет на статью Algorithm в кратчайшем пути из задачи 4.

In [20]:
def fourth_task(graph, link):
    viewed_links = set()
    links = [(0, link)]
    min_lengths = {link: (0, link)}
    while links: 
        current_length, current_link = heappop(links)
        if current_link not in viewed_links:
            for _, next_link, length in graph.neighbor_links.get(current_link, []): 
                if next_link in viewed_links:
                    continue
                if min_lengths.get(next_link) is None:
                    old_min_length = None
                else:
                    old_min_length = min_lengths.get(next_link)[0]
                new_length = current_length + length
                if (old_min_length is None) or (new_length < old_min_length):
                    min_lengths[next_link] = (new_length, current_link)
                    heappush(links, (new_length, next_link))       
            viewed_links.add(current_link)
    return min_lengths

In [21]:
print('Ответ:', fourth_task(weighted_graph, initial_link)[final_link][1])

Ответ: Logic
